# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
finetune = pd.read_csv('data/finetuning_data.csv')

# Parse First Text File

In [2]:
file_path = 'llama_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100)
print(result1)
print(len(df1), total_questions)

Percentage of Questions with Executable Code: 55.425904317386234
1425 2571


In [3]:
#this needs to be rewritten
def check_topic(df):
    count = 0
    for i in range(0, len(df)):
        if df.iloc[i]['topic'] in df.iloc[i]['question']:
            count+=1
            
    return "Percentage of Executable Code that Contains Topic: " + str((count/len(df))*100)
        
    
result2 = check_topic(df1)
result2

'Percentage of Executable Code that Contains Topic: 82.52631578947368'

In [4]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 5.894736842105263


In [5]:
df1 = df1.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df1)):
    question_numbers.append(f"llama_q{i}")
df1['question_number'] = question_numbers
df1_an = df1.sample(n=250, random_state = 42)

In [6]:
df1_an = df1_an[~df1_an['question'].isin(finetune['question'])]
df1 = df1[~df1['question'].isin(finetune['question'])]
df1 = df1[~df1['question'].isin(df1_an['question'])]
extra_an1 = df1.sample(n=250-len(df1_an), random_state = 42)
extra_an1

,question,solution,answer,topic,question_number
1165,The race track has 20 cars. 10 of them are red...,def solution():\n #The race track has 20 ca...,5,cars,llama_q1154
646,Naruto is 16 years old. He is 4 years older t...,def solution():\n #Naruto is 16 years old\n...,12,Naruto,llama_q643
205,A plane travels 1000 miles in 2 hours. How man...,def solution():\n #A plane travels 1000 mil...,2500.0,planes,llama_q205
1011,A car travels 100 miles in 2 hours. It travels...,def solution():\n #The car travels 100 mile...,50.0,cars,llama_q1005
98,Spiderman has 200 spiderwebs. He uses 70% of t...,def solution():\n #Spiderman started with 2...,60.0,Spiderman,llama_q98
1256,A cheerleading squad has 16 girls. 2/3 of the ...,def solution():\n #Total number of girls\n ...,4.0,cheer leading,llama_q1241
593,Harry Potter has 10 galleons. He spends 2 gall...,def solution():\n #Harry Potter has 10 gall...,0,Harry Potter,llama_q591
1088,There are 240 candies in a box. 60 of the cand...,def solution():\n #There are 240 candies in...,130,candy,llama_q1078
668,Spiderman can swing 100 meters in 5 seconds. H...,def solution():\n #Spiderman can swing 100 ...,400.0,Spiderman,llama_q665
443,Steph Curry is the best shooter in the league....,def solution():\n #Steph Curry has 3000 poi...,500,Steph Curry,llama_q443


In [7]:
df1

,question,solution,answer,topic,question_number
0,Barbie has 500 pairs of shoes. 200 are high he...,def solution():\n #Barbie has 500 pairs of ...,150,Barbie,llama_q0
1,Black Panther can run 200 meters in 20 seconds...,def solution():\n #Black Panther can run 20...,0.2,Black Panther,llama_q1
2,A cookie baker has 2500 cookies. \nHe sells 10...,def solution():\n #The baker started with 2...,800,cookies,llama_q2
3,Iron Man has a total of 42 suits. 25 of the su...,def solution():\n #Iron Man has a total of ...,0,Iron Man,llama_q3
5,A pizza has 8 slices. If 2 slices are pepperon...,def solution():\n #Total slices\n total_...,4,pizza,llama_q5
...,...,...,...,...,...
1420,A unicorn has 100 horns. He uses 10 horns to m...,def solution():\n #A unicorn has 100 horns\...,90,unicorns,llama_q1401
1421,150 passengers are boarding a plane. 75 of the...,def solution():\n #150 passengers are board...,65,planes,llama_q1402
1422,LeBron James is a basketball player. He has 25...,def solution():\n #LeBron James has 25 poin...,18,LeBron James,llama_q1403
1423,Captain America is 2 times as strong as Thor. ...,def solution():\n #Captain America is 2 tim...,100,Captain America,llama_q1404


# Parse Second Text File

In [8]:
file_path = 'mathwell_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 66.35177687809268
1475 2223


In [9]:
#Conduct significance test to see whether MATHWELL has a meaningful difference in percentage of executable code after finetuning
#compared to Llama finetuned on GSM8K with code solutions and base Llama
import numpy as np
from scipy import stats
mathwell = np.concatenate([np.ones(1475), np.zeros(2223-1475)])
fine_tuned_llama = np.concatenate([np.ones(2742), np.zeros(4347-2742)])
base_llama = np.concatenate([np.ones(1425), np.zeros(2571-1425)])

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(mathwell, fine_tuned_llama)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(mathwell, base_llama)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 2.6196677957481587
P-value: 0.008821788654780824
T-statistic: 7.763552900356672
P-value: 1.002787084527484e-14


In [10]:
result2 = check_topic(df2)
result2

'Percentage of Executable Code that Contains Topic: 80.8135593220339'

In [11]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 6.440677966101695


In [12]:
df2 = df2.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df2)):
    question_numbers.append(f"mathwell_q{i}")
df2['question_number'] = question_numbers
df2_an = df2.sample(n=250, random_state = 42)
df2_an = df2_an[~df2_an['question'].isin(finetune['question'])]
df2_an

,question,solution,answer,topic,question_number
765,The soccer team has 20 players. \n\n8 of the p...,def solution():\n #The soccer team has 20 p...,7,soccer,mathwell_q741
893,Hello Kitty has 200 pencils. She has 20 more p...,def solution():\n #Hello Kitty has 200 penc...,130,Hello Kitty,mathwell_q865
344,"LeBron James is 6'9"" tall. \nHe's 36 years old...","def solution():\n #LeBron James is 6'9"" tal...",1.38,LeBron James,mathwell_q339
194,Spiderman has 20 spider-bots. He deploys 5 spi...,def solution():\n #Spiderman started with 2...,5,Spiderman,mathwell_q192
205,"Steve, the main character in Minecraft, is in ...",def solution():\n #Steve has 100 health poi...,50.0,Minecraft,mathwell_q203
...,...,...,...,...,...
1402,Hello Kitty has 2000 yen. \n\nShe spends 300 y...,def solution():\n\n # Hello Kitty's initial...,1600,Hello Kitty,mathwell_q1345
1075,100 Fortnite players are in a match. \n25 of t...,def solution():\n #100 Fortnite players are...,50,Fortnite,mathwell_q1037
56,Taylor Swift has 2000 fans. 1000 of them are i...,def solution():\n #Taylor Swift has 2000 fa...,500,Taylor Swift,mathwell_q56
745,"Harry, Ron, and Hermione each have 3 wands. \n...","def solution():\n #Harry, Ron, and Hermione...",12,Harry Potter,mathwell_q721


In [13]:
df2 = df2[~df2['question'].isin(finetune['question'])]
df2 = df2[~df2['question'].isin(df2_an['question'])]
extra_an2 = df2.sample(n=250-len(df2_an), random_state = 42)
extra_an2

,question,solution,answer,topic,question_number
1354,Superman can punch 2000 times per minute. How...,def solution():\n #Superman can punch 2000 ...,20000,Superman,mathwell_q1299
905,A Minecraft world has 10000 blocks. 2000 of th...,def solution():\n #Total blocks\n total_...,3000,Minecraft,mathwell_q876
982,A 25-meter pool is 2.5 times as long as a 10-m...,def solution():\n #A 25-meter pool is 2.5 t...,15,swimming,mathwell_q949
730,A candy store has 1000 lollipops. 200 of them ...,def solution():\n #Total lollipops\n tot...,500,candy,mathwell_q707
511,A bakery bakes 1200 cookies. \n\nThey bake 200...,def solution():\n #The bakery bakes 1200 co...,2100,cookies,mathwell_q503
940,"Patrick Mahomes, the NFL superstar, has 3000 p...",def solution():\n #Patrick Mahomes has 3000...,2000,Patrick Mahomes,mathwell_q908
942,A cat has 9 lives. \n\nA dog has 3 lives.\n\nH...,def solution():\n\n # A cat has 9 lives\n ...,6,cats,mathwell_q910
757,"In the first level of Super Mario, there are 1...","def solution():\n #In the first level, ther...",600,Super Mario,mathwell_q733
450,Serena has 24 Grand Slam titles. 8 of her titl...,def solution():\n #Serena has 24 Grand Slam...,0,Serena Williams,mathwell_q444
102,Thor can summon 10 lightning bolts per second....,def solution():\n #Thor can summon 10 light...,100,"Thor, the God of Thunder",mathwell_q102


In [14]:
df2_copy = df2.drop(df2[df2['question_number'].isin(df2_an['question_number'])].index)
df2_copy.to_csv('data/extra_qs.csv')
df2

,question,solution,answer,topic,question_number
1,Serena Williams has 24 Grand Slam singles titl...,def solution():\n #Serena Williams has 24 G...,2,Serena Williams,mathwell_q1
2,1000 unicorns are in the forest. 200 of them a...,def solution():\n #1000 unicorns are in the...,500,unicorns,mathwell_q2
3,A T-Rex is 20 feet tall. A Brontosaurus is 10...,def solution():\n #A T-Rex is 20 feet tall\...,15.0,dinosaurs,mathwell_q3
4,The lacrosse team has 120 sticks. 20 of the st...,def solution():\n #Total sticks\n sticks...,0,lacrosse,mathwell_q4
5,There are 150 Pokémon in the Kanto region. 50 ...,def solution():\n #Total Pokémon in the Kan...,50,Pokémon,mathwell_q5
...,...,...,...,...,...
1466,A 200-car train is 2 miles long. A 120-car tra...,def solution():\n #A 200-car train is 2 mil...,1.666667,trains,mathwell_q1404
1467,Pikachu has 100 thunderbolts. He uses 20 thund...,def solution():\n #Pikachu has 100 thunderb...,50,Pokémon,mathwell_q1405
1470,A bakery bakes 1200 cookies per day. 300 of th...,def solution():\n #The bakery bakes 1200 co...,500,cookies,mathwell_q1407
1472,Captain Marvel can punch 15 times a second. \n...,def solution():\n #Captain Marvel punches 1...,450,Captain Marvel,mathwell_q1409


# Parse Third Text File

In [15]:
file_path = 'llema_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                if 'while' in solution:
                    continue
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df3 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df3)/total_questions)*100)
print(result1)
print(len(df3), total_questions)

8
Captain America was 1.0 times higher than he was wide.
60
There are at least one cookie left in the cookie jar


How long would you like to use your superpower? a. 1 day; b. 1 week b


Percentage of Questions with Executable Code: 24.32288900690388
916 3766


In [16]:
result2 = check_topic(df3)
result2

'Percentage of Executable Code that Contains Topic: 69.97816593886463'

In [17]:
print(f"Percentage of questions that have 0 as answer: {(len(df3[df3['answer']==0])/len(df3))*100}")

Percentage of questions that have 0 as answer: 4.148471615720524


In [18]:
df3 = df3.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df3)):
    question_numbers.append(f"llema_q{i}")
df3['question_number'] = question_numbers
df3_an = df3.sample(n=250, random_state = 42)

In [19]:
df3

,question,solution,answer,topic,question_number
0,Fred has 4 dogs. 3 of Fred's dogs have 4 legs ...,def solution():\n #Fred had 4 dogs\n dog...,15,dogs,llema_q0
1,At a high school football game there are 5000 ...,def solution():\n #Total number of fans\n ...,2500.0,cheer leading,llema_q1
2,A litter of kittens has 5 kittens. They want t...,def solution():\n #A litter of kittens has ...,10,cats,llema_q2
3,"The Buffalo Bandits have 10 attackers, 15 defe...",def solution():\n #The Buffalo Bandits have...,52,lacrosse,llema_q3
4,"Daddy wants to eat pizza, but not too much. He...",def solution():\n #A slice is 1 piece of pi...,3285,pizza,llema_q4
...,...,...,...,...,...
911,"Captain America has three bags, the red bag, t...",def solution():\n #Captain America has thre...,5,Captain America,llema_q907
912,Serena Williams won 22 Grand Slam titles. The ...,def solution():\n #Serena Williams won 22 G...,4,Serena Williams,llema_q908
913,Superman's heart beats 110 times per minute. H...,def solution():\n #Superman's heart beats 1...,1100,Superman,llema_q909
914,A bakery makes 21 cookies in 3 minutes. How ma...,def solution():\n #Number of cookies the ba...,10500.0,cookies,llema_q910


In [20]:
df3_an = df3_an[~df3_an['question'].isin(finetune['question'])]
df3 = df3[~df3['question'].isin(finetune['question'])]
df3 = df3[~df3['question'].isin(df3_an['question'])]
extra_an3 = df3.sample(n=250-len(df3_an), random_state = 42)
extra_an3

,question,solution,answer,topic,question_number
898,"In the 2013/2014 NBA season, Steph Curry made ...",def solution():\n #In the 2013/2014 NBA sea...,6.5,Steph Curry,llema_q894
297,A football team has 50 cheerleaders. 4 cheerle...,def solution():\n #There are 50 cheerleader...,276,cheer leading,llema_q297
614,Player1 builds 400 \nblocks of diamond blocks....,def solution():\n #Player1 builds 400 block...,-250,Minecraft,llema_q612
460,A high school lacrosse team has scored the fol...,def solution():\n #A high school lacrosse t...,13,lacrosse,llema_q460
106,The Northtown High School Boys Lacrosse Team w...,def solution():\n #A lacrosse team won 50% ...,50.0,lacrosse,llema_q106
730,"At a soccer game, 65 people are wearing team j...",def solution():\n #Number of people wearing...,5,soccer,llema_q728
248,Captain America jumped 4 feet in length and 12...,def solution():\n #Captain America jumped 4...,1.0,Captain America,llema_q248


# Parse Fourth Text File

In [21]:
file_path = 'mammoth_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df4 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df4)/total_questions)*100)
print(result1)
print(len(df4), total_questions)

92
250
16
2
730
22
9
960
24
20
200
150
9
13333.333333333334
325.0
96
24
8
9
86400
16.666666666666668
3
86
10
50.0
5
1
3276
50
64
6
22.369418519393044
24
6000
20000
175
60.0
3000
300
23
300000
9
5.547001962252291
19
16.0
3
2500
5.0
18
240
1350
6.0
6
11990
1000
216.0
750
110
24
1.25
100
95
The coach can divide the team into two groups of 10 players each.
2000
5.0
6
18
37
2000.0
6.666666666666667
3.0
720
22
195
370
5
100
200
144
150
5.5
25.0
30
400
4.0
349
40
8.0
5
10.0
500
625
80
('Pikachu', 'Bulbasaur', 'Charmander')
15.0
47
600.0
2.0
70
2500.0
2
0
1600
480
13
2.0
110
1250.0
33.333333333333336
4.0
21
-1
23.323807579381203
10
40
1.15
24.0
400
21
160
0
24
5
12.5
110
53.333333333333336
4.0
40
18
20.0
10000
4
0.016666666666666666
2.6666666666666665
90
100
4000
7.5
120
240
2
30
96.0
5
36
1000
1
60
100
1.8181818181818181
1000
10.0
37
10
3.0
12
10.0
40.0
8
16777216
1.4
3
5
20
800
20
120
50
51
48.0
5.596
25.0
400
6
20
24
150.0
2.3333333333333335
2
40
37
24
32
10.5
7
22
150
600000
300
1600
120
6

In [22]:
result2 = check_topic(df4)
result2

'Percentage of Executable Code that Contains Topic: 86.9942196531792'

In [23]:
print(f"Percentage of questions that have 0 as answer: {(len(df4[df4['answer']==0])/len(df4))*100}")

Percentage of questions that have 0 as answer: 2.8901734104046244


In [24]:
df4 = df4.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df4)):
    question_numbers.append(f"mammoth_q{i}")
df4['question_number'] = question_numbers
df4_an = df4.sample(n=250, random_state = 42)

In [25]:
df4

,question,solution,answer,topic,question_number
0,Iron Man has 23 suits of armor. He builds 12 m...,def solution():\n #Iron Man has 23 suits of...,31,Iron Man,mammoth_q0
1,Naruto has 100 ninja tools. 20 of them are shu...,def solution():\n #Naruto has 100 ninja too...,50,Naruto,mammoth_q1
2,"Thor, the God of Thunder, can throw his hammer...",def solution():\n #Thor can throw his hamme...,166.666667,"Thor, the God of Thunder",mammoth_q2
3,Iron Man can fly 100 miles per minute. \nHow m...,def solution():\n #Iron Man can fly 100 mil...,500,Iron Man,mammoth_q3
4,Patrick Mahomes is the starting quarterback fo...,def solution():\n #Number of touchdowns thr...,92,Patrick Mahomes,mammoth_q4
...,...,...,...,...,...
341,Naruto has 12 ramen. He eats 3 of them on each...,def solution():\n #Naruto started with 12 r...,6,Naruto,mammoth_q341
342,"If Barbie has 245 1s, 225 5s, 123 10s, and 124...",def solution():\n #Barbie has 245 1s\n o...,5080,candy,mammoth_q342
343,The soccer team has 32 players. Each player ha...,def solution():\n #Number of players\n p...,64,soccer,mammoth_q343
344,Sasuke ate 18 cookies on Monday. He ate 20 coo...,def solution():\n #Sasuke ate 18 cookies on...,60,cookies,mammoth_q344


In [26]:
df4_an = df4_an[~df4_an['question'].isin(finetune['question'])]
df4 = df4[~df4['question'].isin(finetune['question'])]
df4 = df4[~df4['question'].isin(df4_an['question'])]
extra_an4 = df4.sample(n=250-len(df4_an), random_state = 42)
extra_an4

,question,solution,answer,topic,question_number
260,3 dogs ate 12 bones. How many bones did each d...,def solution():\n #3 dogs ate 12 bones\n ...,4.0,dogs,mammoth_q260
71,Harry Potter has 3000 vibranium-powered punche...,def solution():\n #Harry Potter started wit...,2300,Harry Potter,mammoth_q71


## Parse GPT-3.5 Generations

In [27]:
file_path = 'gpt35_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("Question:")[0].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df5 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df5)/total_questions)*100)
print(result1)
print(len(df5), total_questions)

Percentage of Questions with Executable Code: 97.54269054560599
2342 2401


In [28]:
result2 = check_topic(df5)
result2

'Percentage of Executable Code that Contains Topic: 86.12297181895816'

In [29]:
print(f"Percentage of questions that have 0 as answer: {(len(df5[df5['answer']==0])/len(df5))*100}")

Percentage of questions that have 0 as answer: 1.5798462852263024


In [30]:
df5 = df5.drop_duplicates(subset='question')
df5 = df5[~df5['question'].isin(finetune['question'])]
question_numbers = []
for i in range(0, len(df5)):
    question_numbers.append(f"gpt35_q{i}")
df5['question_number'] = question_numbers
df5_an = df5.sample(n=250, random_state = 42)

In [31]:
df5

,question,solution,answer,topic,question_number
0,The Power Rangers are trying to defeat a monst...,def solution():\n #Each Ranger has a weapon...,3000,Power Rangers,gpt35_q0
1,There are 6 players on a volleyball team. In t...,def solution():\n #There are 6 players on t...,414,volleyball,gpt35_q1
2,A plane is flying at a speed of 500 miles per ...,def solution():\n #The plane has been flyin...,1500,planes,gpt35_q2
3,"There are 4 Ninja Turtles: Michelangelo, Donat...",def solution():\n #There are 4 Ninja Turtle...,56,Ninja Turtles,gpt35_q3
4,Naruto can run 5 miles in 30 minutes. How many...,def solution():\n #Naruto can run 5 miles i...,10.0,Naruto,gpt35_q4
...,...,...,...,...,...
2337,Captain America has 8 shields. Each shield has...,def solution():\n #Captain America has 8 sh...,25.13272,Captain America,gpt35_q2334
2338,The Wildcats football team has scored 120 poin...,def solution():\n #The Wildcats have scored...,240.0,football,gpt35_q2335
2339,The cheerleading squad has 20 members. The squ...,def solution():\n #The cheerleading squad h...,10.0,cheer leading,gpt35_q2336
2340,"In Fortnite, there are 100 players on the isla...",def solution():\n #Total number of players ...,40,Fortnite,gpt35_q2337


In [32]:
df5_an

,question,solution,answer,topic,question_number
637,Barbie has 1000 outfits in her closet. She nee...,def solution():\n #Barbie has 1000 outfits ...,995,Barbie,gpt35_q637
1299,"In a volleyball tournament, there are 8 teams....",def solution():\n #There are 8 teams in the...,28.0,volleyball,gpt35_q1298
387,The local lacrosse team has 18 players. During...,def solution():\n #The local lacrosse team ...,0.833333,lacrosse,gpt35_q387
2070,Black Panther has 1500 vibranium shards. He ne...,def solution():\n #Black Panther has 1500 v...,300.0,Black Panther,gpt35_q2069
247,The Mighty Morphin Power Rangers have a total ...,def solution():\n #The Power Rangers have 3...,150,Power Rangers,gpt35_q247
...,...,...,...,...,...
605,"In Minecraft, a player needs 3 blocks of iron ...","def solution():\n #To craft one bucket, 3 b...",105,Minecraft,gpt35_q605
237,The Golden State Warriors have played 70 games...,def solution():\n #The Warriors have played...,64.285714,basketball,gpt35_q237
51,"There are 5 bags of candy, each with a differe...",def solution():\n # Number of pieces in eac...,65,candy,gpt35_q51
1936,"In a Pokémon tournament, Ash has 6 Pikachu, 4 ...","def solution():\n #Ash has 6 Pikachu, 4 Cha...",520,Pokémon,gpt35_q1935


## Parse GPT-4 Generations

In [33]:
file_path = 'gpt4_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("Question:")[0].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("```python")[1].strip()
            solution = solution.split("```")[0].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df6 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df6)/total_questions)*100)
print(result1)
print(len(df6), total_questions)

3.3333333333333335
112
110
The opposing team scored 1 goal(s).
15
The paleontologist discovered a total of 255 horns and teeth.
Black Panther has 35 Vibranium shards left.
There are 10 slices of pizza left.
300
Sarah has 6 cookies left.
Taylor Swift sold a total of 45000 tickets for all three concerts.
29
The team won 10 games and lost 5 games.
Number of Plane C flights needed: 2
27
Mario needs to defeat 7 Goombas.
The soccer team won 8 games.
50
The two trains will meet after 2.6923076923076925 hours.
20.0
20
Wonder Woman has 85 magical bracelets now.
62.5
30
The two trains will be 40 miles apart after the faster train has been running for two hours.
70
3
10
Number of unicorns in the meadow: 71
25
Each unicorn has 8.0 crystals.
840
Jamie has 30 gold coins left at the end of the game.
112
Sarah scored a total of 52 points in the three games.
200
350
Patrick Mahomes threw for 180 yards that were not for touchdowns.
42
33
The plane carried a total of 230 passengers on Monday and Tuesday.

In [34]:
result2 = check_topic(df6)
result2

'Percentage of Executable Code that Contains Topic: 83.13725490196079'

In [35]:
print(f"Percentage of questions that have 0 as answer: {(len(df6[df6['answer']==0])/len(df6))*100}")

Percentage of questions that have 0 as answer: 0.39215686274509803


In [36]:
df6 = df6.drop_duplicates(subset='question')
df6 = df6[~df6['question'].isin(finetune['question'])]
question_numbers = []
for i in range(0, len(df6)):
    question_numbers.append(f"gpt4_q{i}")
df6['question_number'] = question_numbers
df6_an = df6.sample(n=250, random_state = 42)

In [37]:
df6

,question,solution,answer,topic,question_number
0,"In a field hockey tournament, each team plays ...",def solution():\n # There are 8 teams in th...,12,field hockey,gpt4_q0
1,Superman helps a construction company by movin...,def solution():\n # Superman can carry 10 b...,3,Superman,gpt4_q1
2,"A volleyball team won 15 games this season, wh...",def solution():\n # The team won 15 games t...,5.0,volleyball,gpt4_q2
3,Five Power Rangers are each fighting an equal ...,def solution():\n # There are a total of 35...,7.0,Power Rangers,gpt4_q3
4,The Power Rangers are fighting a monster and n...,def solution():\n # Each Zord consumes 250 ...,750,Power Rangers,gpt4_q4
...,...,...,...,...,...
250,Wonder Woman is helping to build a new trainin...,def solution():\n #The total number of bric...,105,Wonder Woman,gpt4_q250
251,Sally has 42 pieces of candy. She gives 12 pie...,def solution():\n #Sally starts with 42 pie...,60,candy,gpt4_q251
252,Captain Marvel is helping to distribute energy...,def solution():\n # Captain Marvel starts w...,50,Captain Marvel,gpt4_q252
253,Hello Kitty has 10 red bows and 15 pink bows. ...,def solution():\n # Hello Kitty starts with...,17,Hello Kitty,gpt4_q253


In [38]:
df6_an

,question,solution,answer,topic,question_number
190,Jenny has 12 cats. She decides to buy more cat...,def solution():\n # Jenny has 12 cats\n ...,4,cats,gpt4_q190
6,"In a soccer tournament, the Eagles scored twic...",def solution():\n # The Tigers scored 12 go...,14,soccer,gpt4_q6
79,Maria baked 48 cookies to share with her frien...,def solution():\n #Maria started with 48 co...,13,cookies,gpt4_q79
205,Two trains are traveling towards each other on...,def solution():\n # Speed of Train A\n s...,4.0,trains ###,gpt4_q205
117,Barbie has a collection of dresses. She has 7 ...,def solution():\n # Barbie has 7 red dresse...,11,Barbie,gpt4_q117
...,...,...,...,...,...
121,Iron Man is building new suits for his superhe...,def solution():\n # Initial number of suits...,26,Iron Man,gpt4_q121
254,Super Mario collects 100 coins and gets an ext...,def solution():\n # Super Mario starts with...,7,Super Mario,gpt4_q254
20,"Samantha has a dog named Max, who loves to col...",def solution():\n # Max finds 3 sticks ever...,5,dogs,gpt4_q20
188,Hello Kitty has 5 apples. She gives 2 apples t...,def solution():\n # Hello Kitty starts with...,7,Hello Kitty,gpt4_q188


## Parse Llama Prompted with GSM8K

In [60]:
file_path = 'llama_questions_gsm_prompted.txt'
data = []
import sys
from io import StringIO
with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            initial_vars = dict(locals().items())
            old_stdout = sys.stdout
            redirected_output = sys.stdout = StringIO()
            try:
                if 'print(' in solution:
                    exec(solution, globals(), locals())
                    sys.stdout = old_stdout

                    value = redirected_output.getvalue().strip()
                    data.append({"question": question, "solution": solution, 'answer': value, 'topic': topic})

                    # Create a dictionary to store the variables after execution
                    vars_after_exec = dict(locals().items())

                    # Find the new variables created by exec()
                    new_variables = {var_name: var_value for var_name, var_value in vars_after_exec.items() if var_name not in initial_vars}

                    # Delete the new local variables
                    for var_name in new_variables:
                        del locals()[var_name]

            except:
                pass
        except:
            pass
df7 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df7)/total_questions)*100)
print(result1)
print(len(df7), total_questions)

In [61]:
len(df7), total_questions

(206, 509)

In [62]:
"Percentage of Questions with Executable Code: " + str((len(df7)/total_questions)*100)

'Percentage of Questions with Executable Code: 40.47151277013752'

In [63]:
f"Percentage of questions that have 0 as answer: {(len(df7[df7['answer']==0])/len(df7))*100}"

'Percentage of questions that have 0 as answer: 0.0'

In [67]:
df7 = df7.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df7)):
    question_numbers.append(f"llama_gsm_q{i}")
df7['question_number'] = question_numbers
df7_an = df7.sample(n=min(100, len(df7)), random_state = 42)

In [68]:
df7

,question,solution,answer,topic,question_number
0,A dog named Max is 10 inches taller than the a...,# define the average dog's height\navg_height ...,30,dogs,llama_gsm_q0
1,,# define the variables\nice_cream_flavors = 6\...,18,ice cream,llama_gsm_q1
2,The soccer team has 3 games per week. Each gam...,# define the number of games per week and the ...,11,soccer,llama_gsm_q2
3,A Minecraft world is 256 blocks wide and 256 b...,# define the variables\nwidth = 256\nheight = ...,65536,Minecraft,llama_gsm_q3
4,Barbie has 200 dolls. She gives away 5 dolls e...,# define the total number of dolls\ntotal_doll...,95,Barbie,llama_gsm_q4
...,...,...,...,...,...
201,A field hockey game is played between two team...,# define the constants\nfeet_per_yard = 3\nlen...,54000,field hockey,llama_gsm_q201
202,Taylor Swift has a collection of 1000 music al...,# define the number of albums from the 80s and...,1800,Taylor Swift,llama_gsm_q202
203,"You have 3 Pokemon with 200 health points, 1 P...",# define the variables\npokemon_1 = 200\npokem...,-350,Pokémon,llama_gsm_q203
204,A cheerleading squad has 10 members. The squad...,"# define the number of cartwheels, jumps, and ...",250,cheer leading,llama_gsm_q204


In [71]:
df7_an
df7_an.to_csv('data/llama_gsm_questions.csv')

## MATHWELL Trained on EGSM Only

In [25]:
file_path = "mathwell_questions_egsm_only.txt"
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
egsm_only = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(egsm_only)/total_questions)*100)
print(result1)
print(len(egsm_only), total_questions)

In [26]:
result1

'Percentage of Questions with Executable Code: 53.01418439716312'

In [27]:
len(egsm_only), total_questions

(299, 564)

In [28]:
f"Percentage of questions that have 0 as answer: {(len(egsm_only[egsm_only['answer']==0])/len(egsm_only))*100}"

'Percentage of questions that have 0 as answer: 10.033444816053512'

In [29]:
egsm_only = egsm_only.drop_duplicates(subset='question')
egsm_only = egsm_only[~egsm_only['question'].isin(finetune['question'])]
question_numbers = []
for i in range(0, len(egsm_only)):
    question_numbers.append(f"egsm_only_q{i}")
egsm_only['question_number'] = question_numbers
egsm_only_an = egsm_only.sample(min(100, len(egsm_only)), random_state = 42)

In [30]:
len(egsm_only)

233

In [31]:
egsm_only_an

,question,solution,answer,topic,question_number
105,Barbie has 2000 shoes. 1000 of the shoes are h...,def solution():\n #Barbie has 2000 shoes\n ...,300,Barbie,egsm_only_q84
277,Captain America can punch 20 times in 10 secon...,def solution():\n #Captain America can punc...,120.0,Captain America,egsm_only_q216
297,Wonder Woman has 200 lassos. \n\nShe lassos 10...,def solution():\n #Wonder Woman has 200 las...,50,Wonder Woman,egsm_only_q231
10,"Patrick Mahomes, the quarterback for the Kansa...","def solution():\n #Patrick Mahomes has 50,0...",120000,Patrick Mahomes,egsm_only_q9
157,Super Mario has 2000 coins. He spends 500 coin...,def solution():\n #Super Mario started with...,1100,Super Mario,egsm_only_q126
...,...,...,...,...,...
178,The field hockey team has 20 players. 10 of th...,def solution():\n #The field hockey team ha...,5,field hockey,egsm_only_q141
96,Batman can punch 20 times per minute. He fight...,def solution():\n #Batman can punch 20 time...,100,Batman,egsm_only_q78
171,Naruto has 300 ryo. He spends 100 ryo on a new...,def solution():\n #Naruto started with 300 ...,0,Naruto,egsm_only_q136
29,Barbie has 1000 pink outfits. 200 of them are ...,def solution():\n #Barbie has 1000 pink out...,0,Barbie,egsm_only_q26


In [19]:
#egsm_only_an.to_csv('data/egsm_only_questions.csv')

## Calculate Standard Deviations

In [20]:
mathwell = np.concatenate([np.ones(1475), np.zeros(2223-1475)])
print(f"Mathwell SD: {np.std(mathwell)}")
print(f"Mathwell SE: {st.sem(mathwell)*100}")

base_llama = np.concatenate([np.ones(1425), np.zeros(2571-1425)])
print(f"Llama SD: {np.std(base_llama)}")
print(f"Llama SE: {st.sem(base_llama)*100}")

llema = np.concatenate([np.ones(916), np.zeros(3766-916)])
print(f"Llema SD: {np.std(llema)}")
print(f"Llema SE: {st.sem(llema)*100}")
                  
mammoth = np.concatenate([np.ones(346), np.zeros(5025-346)])
print(f"Mammoth SD: {np.std(mammoth)}")
print(f"Mammoth SE: {st.sem(mammoth)*100}")

mammoth = np.concatenate([np.ones(346), np.zeros(5025-346)])
print(f"Mammoth SD: {np.std(mammoth)}")
print(f"Mammoth SE: {st.sem(mammoth)*100}")

gpt4 = np.concatenate([np.ones(255), np.zeros(382-255)])
print(f"gpt4 SD: {np.std(gpt4)}")
print(f"gpt4 SE: {st.sem(gpt4)*100}")

gpt35 = np.concatenate([np.ones(2342), np.zeros(2401-2342)])
print(f"gpt35 SD: {np.std(gpt35)}")
print(f"gpt35 SE: {st.sem(gpt35)*100}")

llama_gsm = np.concatenate([np.ones(206), np.zeros(509-206)])
print(f"llama_gsm SD: {np.std(llama_gsm)}")
print(f"llama_gsm SE: {st.sem(llama_gsm)*100}")

egsm_only = np.concatenate([np.ones(299), np.zeros(564-299)])
print(f"egsm_only SD: {np.std(egsm_only)}")
print(f"egsm_only SE: {st.sem(egsm_only)*100}")

In [23]:
f"egsm_only SE: {st.sem(egsm_only)*100}"

'egsm_only SE: 2.103416274078959'

# Concat DFs

In [62]:
# df = pd.concat([df1, df2, df3, df4, df5, df6])
# df = df.sample(frac = 1, random_state = 42)
# df_an = pd.concat([df1_an, df2_an, df3_an, df4_an, df5_an, extra_an1, extra_an2, extra_an3, extra_an4])
# extra_an = pd.concat([extra_an1, extra_an2, extra_an3, extra_an4])
df_an = pd.read_csv('data/zooniverse_model_questions.csv')
df_an = pd.concat([df_an, df5_an, df6_an])
df_an = df_an.drop_duplicates(subset = "question_number")
df_an = df_an.sample(frac=1, random_state = 42)
#df.to_csv("data/model_questions.csv")
df_an.to_csv("data/zooniverse_model_questions.csv")

In [63]:
df_an['question_number'].value_counts()

question_number
llema_q603       1
llema_q494       1
gpt4_q203        1
gpt4_q19         1
llama_q953       1
                ..
llema_q534       1
mammoth_q211     1
mathwell_q509    1
gpt4_q82         1
gpt4_q149        1
Name: count, Length: 1500, dtype: int64

df_an.drop_duplicates(subset = 'question')

# Send Each Question to .txt file for Zooniverse Upload

for i in range(0, len(egsm_only_an)):
    question_number = egsm_only_an.iloc[i]['question_number']
    question = egsm_only_an.iloc[i]['question']
    solution = egsm_only_an.iloc[i]['solution']
    answer = str(egsm_only_an.iloc[i]['answer'])
    topic = egsm_only_an.iloc[i]['topic']
    file_name = f"{question_number}.txt"
    file_path = "Zooniverse/egsm_only/"+file_name
    text = "Topic: "+ topic + "\n\nQuestion:\n" + question + "\n\nSolution:\n" + solution + "\n\nAnswer: " + answer
    with open(file_path, "w") as f:  # Open the file in append mode ("a")
        f.write(text)  # Append the newly generated text to the file